# Debt Collector Data Parser

Excel to CSV Debt Collector Database and more

This file converts all other files in this database to two usable files for the repo.

Note: Multiple CSV files are broken and have been handled in the code where necessary.

By Andrew Woska

## Data sources:

mass.gov (updates quarterly)
- [Current](https://www.mass.gov/doc/debt-collector-licensee-list/download)
- [Archives](https://web.archive.org/web/20190515000000*/https://www.mass.gov/lists/download-a-list-of-approved-licensees)
- [Download Archives](https://web.archive.org/web/20210401000000*/https://www.mass.gov/doc/debt-collector-licensee-list/download)

ISIN RMAI Roster
- [Roster](https://docs.google.com/spreadsheets/d/1fc34K8nZVLFaZ2DSf2jAzAppkjumD2g7/edit?usp=drive_link&ouid=111837632356278279925&rtpof=true&sd=true)
- [Debt Collector Annual Report](https://docs.google.com/spreadsheets/d/1HpjLDZkyYS1-9QIcP5CN3FqbIG9Rc_YTy4GdHt-TpHo/edit?usp=drive_link)
- [RMAI Certified Businesses](https://rmaintl.org/certification-education/certified-receivables-business/certified-company-search/)
  - Currently only Collection Agencies completed for implementation purposes
- [RMAI Certified Individuals](https://rmaintl.org/certification-education/individual-certification-crcp/certified-individual-search/)
  - Not considered in dataset below since many are not affiliated with debt collection companies

Should be run every quarter.\
There are previous archives missing in the current dataset. Feel free to add more and run this file again. (last updated: 05-01-2024)

In [7]:
import pandas as pd
import numpy as np
import glob

In [8]:
# get all file names in the directory
fp = glob.glob('debt-collector-licensee-[0-9][0-9]-20[0-9][0-9].xlsx')
cfp = glob.glob('debt-collector-licensee-[0-9][0-9]-20[0-9][0-9].csv')
isin_roster = "Receivables Management Association International Membership List (Debt Buyers).xlsx"
isin_annual = "DebtCollectorAnnualReport_Data final.xlsx"
rmai_business = "rmai_certified_businesses.xlsx"
rmai_individual = "rmai_certified_individuals.xlsx" # not used yet
fp, cfp, isin_roster, isin_annual, rmai_business, rmai_individual

(['debt-collector-licensee-01-2018.xlsx',
  'debt-collector-licensee-01-2021.xlsx',
  'debt-collector-licensee-04-2018.xlsx',
  'debt-collector-licensee-04-2021.xlsx',
  'debt-collector-licensee-06-2023.xlsx',
  'debt-collector-licensee-07-2018.xlsx',
  'debt-collector-licensee-09-2017.xlsx',
  'debt-collector-licensee-09-2020.xlsx',
  'debt-collector-licensee-12-2023.xlsx'],
 ['debt-collector-licensee-01-2019.csv',
  'debt-collector-licensee-01-2021.csv',
  'debt-collector-licensee-03-2020.csv',
  'debt-collector-licensee-04-2019.csv',
  'debt-collector-licensee-06-2019.csv',
  'debt-collector-licensee-06-2020.csv',
  'debt-collector-licensee-09-2019.csv',
  'debt-collector-licensee-09-2020.csv',
  'debt-collector-licensee-12-2019.csv'],
 'Receivables Management Association International Membership List (Debt Buyers).xlsx',
 'DebtCollectorAnnualReport_Data final.xlsx',
 'rmai_certified_businesses.xlsx',
 'rmai_certified_individuals.xlsx')

In [30]:
annual = pd.read_excel(isin_annual)
annual = annual.rename(columns={'Licensee Name': 'Company Name'})
annual = annual.rename(columns={'NMLS Number': 'License Number'})
annual = annual[['Company Name', 'License Number', 'Email Address']]
# append the annual report with the dataframe where columns match
annual

,Company Name,License Number,Email Address
0,"LoanCare, LLC",2916,licensing@loancare.net
1,NewRez LLC,3013,blittle@Newrez.com
2,"PennyMac Loan Services, LLC",35953,regrelations@pnmac.com
3,PHH Mortgage Corporation,2726,Sheri.Robinson@mortgagefamily.com
4,"Servis One, Inc.",38078,jbalmer@bsifinancial.com
...,...,...,...
424,Innovate Loan Servicing Corporation,1042702,compliance@innovateauto.com
425,Ocwen Financial Solutions Private Limited,15877,Ravi.sp@ocwen.com
426,LendingPoint LLC,1424139,licenses@lendingpoint.com
427,"Encore Receivable Management, Inc.",181265,Erin.bonta-lewis@concentrix.com


In [9]:
# read all files into a list of dataframes
dfs = [pd.read_excel(f) for f in fp]

# remove first row of each dataframe and set second row as header
for df in dfs:
    df.columns = df.iloc[0]
    df = df[1:]

# remove certain CSVs if it exists since error reading it
try:
    cfp.remove('debt-collector-licensee-01-2019.csv')
except:
    pass
try:
    cfp.remove('debt-collector-licensee-04-2019.csv')
except:
    pass
dfs += [pd.read_csv(f) for f in cfp]
# concatenate all dataframes into one with same columns
df = pd.concat(dfs, axis=0, ignore_index=True)
# remove columns Branch ID, Street Address
df = df.drop(columns=['Branch ID', 'Street Address', 'Contact Phone Number'])

# read in the roster and annual report
roster = pd.read_excel(isin_roster)
# remove first row of roster and set second row as header
roster.columns = roster.iloc[0]
roster = roster[1:]
roster = roster.rename(columns={'Physical State': 'State'})
roster = roster.drop(columns=['Membership Type'])
# concatenate the roster with the dataframe where columns match
df = pd.concat([df, roster], axis=0, ignore_index=True)


# read in the annual report
annual = pd.read_excel(isin_annual)
annual = annual.rename(columns={'Licensee Name': 'Company Name'})
annual = annual.rename(columns={'NMLS Number': 'Company ID'})
annual = annual[['Company Name', 'Company ID', 'Email Address']]
# add annual report to the dataframe by rows
df = pd.concat([df, annual], axis=0, ignore_index=True)

# read in the RMAI certified businesses
rmai_bus = pd.read_excel(rmai_business)
rmai_bus = rmai_bus[rmai_bus['Type'].isin(['Debt Buyer', 'Collection Agency'])]
rmai_bus = rmai_bus.rename(columns={'Business Name': 'Company Name'})
rmai_bus = rmai_bus.drop(columns=['Type'])
df = pd.concat([df, rmai_bus], axis=0, ignore_index=True)

# read in the RMAI certified individuals
rmai_ind = pd.read_excel(rmai_individual)
# TODO: add companies to the dataframe when it is determined how

# delete all rows that match column row
df = df[df['Company ID'] != 'Company ID']

# group Company Name alphabetically
df = df.sort_values('Company Name')

df = df.drop_duplicates()
df = df.dropna(how='all')

# if State is not null, assume Country is USA
df['Country'] = np.where(df['State'].notnull(), 'USA', df['Country'])

# create all inclusive csv file
df.to_csv('debt-collector-licensee-all.csv', index=False)
df.head()

,Company ID,Company Name,City,State,Country,Postal Code,Contact Name,Contact Email,License Number,License Name,Email Address
12704,950663,"Media Collections, Inc. dba Joseph, Mann & Creed",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bill.mann@JMCBiz.com
4458,2280,21St Mortgage Corporation,Knoxville,TN,USA,37902,Rachel Cupp,rachelcupp@21stmortgage.com,DC2280,MA Debt Collector,NaN
1,2280,21st Mortgage Corporation,Knoxville,TN,USA,37902,Rachel Cupp,rachelcupp@21stmortgage.com,DC2280,MA Debt Collector,NaN
10487,2280,21st Mortgage Corporation,Knoxville,TN,USA,37932,Kennith Nunez,licensingupdates@21stmortgage.com,DC2280,MA Debt Collector,NaN
10486,2280,21st Mortgage Corporation,Knoxville,TN,USA,37902,Kennith Nunez,licensingupdates@21stmortgage.com,DC2280,MA Debt Collector,NaN


In [10]:
# if Company ID is repeated, only keep the first one
# cids = df['Company ID'].unique()
# cp_df = df.copy()
# for cid in cids:
#     if not pd.isnull(cid):
#         cp_df = cp_df.drop_duplicates(subset='Company Name', keep='first').reset_index(drop=True)

# remove all duplicate rows of Company Name

# TODO: Figure out how to consider companies such as the example below where they are all the same company
# ABC, LTD
# A.B.C., LTD
# A B C, LTD

# loop through all rows and remove all rows that have the same Company Name
# and keep the first one
cp_df = df.copy()
# add a column Suspected Name to the dataframe
cp_df['Suspected Name'] = ''
for i, row in df.iterrows():
    comp = row['Company Name']
    comp = comp.title()
    # if company name starts with a non alphanumeric character, remove it
    while not comp[0].isalnum():
        comp = comp[1:]
    # get rid of all parentheses and what's inside
    if '(' in comp:
        comp = comp[:comp.index('(')]
    # # get rid of all brackets and what's inside
    if '[' in comp:
        comp = comp[:comp.index('[')]
    # replace all commas with nothing
    if ',' in comp:
        comp = comp.replace(',', ' ')
    # get rid of all periods
    while '.' in comp:
        comp = comp.replace('.', '')
    # get rid of everything after Inc, LLC, etc.
    if ' Inc' in comp:
        comp = comp[:comp.index('Inc')]
        comp += ' Inc'
    elif ' Ltd' in comp:
        comp = comp[:comp.index('Ltd')]
        comp += ' Ltd'
    elif ' Llc' in comp:
        comp = comp[:comp.index('Llc')]
        comp += ' Llc'
    elif ' Plc' in comp:
        comp = comp[:comp.index('Plc')]
        comp += ' Plc'
    
    # enforce one space between words
    comp = ' '.join(comp.split())
    
    if i == 0:
        continue
    if comp in cp_df['Suspected Name'].values:
        cp_df = cp_df.drop(i)
    else:
        # add row Suspected Name to the dataframe
        cp_df.loc[i, 'Suspected Name'] = comp

# create all filtered csv file
cp_df.to_csv('debt-collector-licensee-filtered.csv', index=False)
cp_df.head()

,Company ID,Company Name,City,State,Country,Postal Code,Contact Name,Contact Email,License Number,License Name,Email Address,Suspected Name
12704,950663,"Media Collections, Inc. dba Joseph, Mann & Creed",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bill.mann@JMCBiz.com,Media Collections Inc
4458,2280,21St Mortgage Corporation,Knoxville,TN,USA,37902,Rachel Cupp,rachelcupp@21stmortgage.com,DC2280,MA Debt Collector,NaN,21St Mortgage Corporation
12035,NaN,"640 Financial Group, LLC",NaN,IL,USA,NaN,NaN,NaN,NaN,NaN,NaN,640 Financial Group Llc
2216,934928,A.R.C. Accounts Recovery (U.S.A.) Corporation LLC,Kelowna,NaN,BCC,V1X 1Y2,Mark Ball,Mark.Ball@aro.ca,DC0830,MA Debt Collector,NaN,ARC Accounts Recovery
2217,937594,"A.R.M. Solutions, Inc.",Camarillo,CA,USA,93012,Mellissa Sinutko,compliance@armsolutions.com,DC937594,MA Debt Collector,NaN,ARM Solutions Inc


In [11]:
print('Full dataset number of rows:', df.shape[0])
print('Filtered dataset number of rows:', cp_df.shape[0])

Full dataset number of rows: 4835
Filtered dataset number of rows: 1157
